# Kabuto Auto Trader - パラメータ最適化

このノートブックでは、実トレード結果を分析して、Pine Scriptのパラメータを最適化します。

## 最適化フロー
1. 実トレードデータを読み込み
2. パフォーマンスを分析
3. 問題を診断
4. 推奨パラメータを計算
5. Pine Scriptコードを生成
6. TradingViewで再テスト

In [ ]:
# ライブラリインポート
import sys
sys.path.append('../lib')

import pandas as pd
import numpy as np
from datetime import datetime

from data_loader import KabutoDataLoader
from analytics import PerformanceAnalyzer
from optimizer import ParameterOptimizer

print("ライブラリのインポート完了")

## 1. データ読み込み

In [ ]:
# オプション1: Excelファイルから読み込み（実データ）
# EXCEL_PATH = '../../Kabuto Auto Trader.xlsm'
# loader = KabutoDataLoader(excel_path=EXCEL_PATH)
# trades = loader.load_all_trades(source='excel')

# オプション2: サンプルデータを使用（テスト用）
trades = KabutoDataLoader.generate_sample_data(n_trades=100)

print(f"読み込んだ取引数: {len(trades)}")
print(f"期間: {trades['timestamp'].min()} - {trades['timestamp'].max()}")
trades.head()

## 2. 現在のパフォーマンス分析

In [ ]:
# パフォーマンス分析
analyzer = PerformanceAnalyzer(trades)
analyzer.print_report()

## 3. パラメータ最適化

In [ ]:
# パラメータ最適化器を初期化
optimizer = ParameterOptimizer(trades)

# 最適化レポート出力
optimizer.print_optimization_report()

## 4. 推奨パラメータ詳細

In [ ]:
# 推奨パラメータ取得
recommendations = optimizer.recommend_parameters()

if recommendations:
    print("推奨パラメータ一覧:\n")
    for param_name, param_info in recommendations.items():
        print(f"【{param_name}】")
        print(f"  現在値: {param_info['current']}")
        print(f"  推奨値: {param_info['recommended']}")
        print(f"  理由: {param_info['reason']}")
        print()
else:
    print("✅ 現在のパラメータで問題ありません")

## 5. Pine Script コード生成

以下のコードをコピーして、TradingViewに貼り付けてください。

In [ ]:
# Pine Scriptコード生成
pine_script = optimizer.generate_pine_script()

print(pine_script)

# ファイルに保存（オプション）
# optimizer.save_pine_script('../reports/optimized_parameters.pine')
# print("\nPine Scriptを ../reports/optimized_parameters.pine に保存しました")

## 6. TradingViewでの適用手順

### ステップ1: コードをコピー
上記のPine Scriptコードをコピーしてください。

### ステップ2: TradingViewを開く
1. TradingViewにログイン
2. Pine エディタを開く
3. `kabuto_strategy_v1.pine` を開く

### ステップ3: パラメータ部分を置き換え
1. 20-46行目の「入力パラメータ」セクションを選択
2. 上記でコピーしたコードに置き換え
3. 保存（Ctrl+S）

### ステップ4: バックテスト実行
1. Strategy Testerタブを開く
2. テスト期間を設定（例: 過去6ヶ月）
3. 実行

### ステップ5: 結果確認
以下の指標を確認：
- **勝率**: 50%以上が目標
- **プロフィットファクター**: 1.5以上が目標
- **最大ドローダウン**: 50,000円以内が目標
- **シャープレシオ**: 1.0以上が目標

改善が確認できたら、本番環境に適用してください。

## 7. カスタムパラメータでテスト（高度な使い方）

推奨値以外のパラメータを試したい場合：

In [ ]:
# カスタムパラメータ設定
custom_params = {
    'emaFastPeriod': 7,
    'emaMediumPeriod': 30,
    'rsiLower': 60,
    'atrTpMultiplier': 6.0,
    'maxDailyEntries': 2
}

# Pine Script生成
custom_script = optimizer.generate_pine_script(params=custom_params)
print(custom_script)

## 8. 複数パラメータセットの比較（将来拡張）

複数のパラメータセットを試して最適なものを選ぶ場合：

In [ ]:
# パラメータセット候補
param_sets = [
    {'name': '保守的', 'maxDailyEntries': 1, 'atrSlMultiplier': 1.5, 'minRrRatio': 2.5},
    {'name': '標準', 'maxDailyEntries': 3, 'atrSlMultiplier': 2.0, 'minRrRatio': 1.5},
    {'name': '積極的', 'maxDailyEntries': 5, 'atrSlMultiplier': 2.5, 'minRrRatio': 1.0},
]

print("パラメータセット候補:\n")
for param_set in param_sets:
    print(f"【{param_set['name']}】")
    print(f"  1日最大エントリー数: {param_set['maxDailyEntries']}")
    print(f"  SL倍率: {param_set['atrSlMultiplier']}")
    print(f"  最低RR比: {param_set['minRrRatio']}")
    print()

print("それぞれのパラメータセットをTradingViewでテストして、最適なものを選択してください。")

## 9. まとめ

このノートブックでは、実トレードデータから最適なパラメータを推奨しました。

### 次のステップ
1. ✅ 推奨パラメータをTradingViewで適用
2. ✅ バックテストで改善を確認
3. ✅ 改善が確認できたら本番環境に適用
4. ✅ 1-2週間後に再度このノートブックで分析
5. ✅ 継続的に最適化を繰り返す

### 注意事項
- パラメータ変更後は必ずバックテストで検証してください
- 過度な最適化（オーバーフィッティング）に注意
- 最低100回以上の取引データで分析することを推奨
- フォワードテストで実際の市場での性能を確認してください